# 오늘 해야할 일!
* 데이터를 -21.04 : train, 21.05-06 : test 분할하기
* 책에서 나온 모델 적용시키기
* 평가 지표 활용해서 모델 뭐가 나은지 비교하기!

In [ ]:
!pip install tzlocal

In [ ]:
!pip install rpy2

In [ ]:
!pip install pmdarima==1.5.3

     |████████████████████████████████| 1.5 MB 29.3 MB/s 


In [ ]:
import tzlocal
import rpy2
import pmdarima
import statsmodels

In [ ]:
import pandas as pd
import numpy as np
import datetime

# 시계열 예측(Time Series Forecasting)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.install_packages('forecast')
utils.install_packages('forecastHybrid')

In [ ]:
data2 = pd.read_csv('/content/drive/MyDrive/21 세미프로젝트_2차 (1)/데이터/01_음식물쓰레기_기본수정.csv',encoding='cp949')
data2

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
0,2018-01-01,제주시,50110670,외도동,W6X062,2,15500,464
1,2018-01-01,제주시,50110630,아라동,W6XA97,25,59800,1787
2,2018-01-01,제주시,50110630,아라동,W6XA96,8,25350,758
3,2018-01-01,제주시,50110630,아라동,W6XA95,12,30000,898
4,2018-01-01,제주시,50110630,아라동,W6XA94,14,44050,1317
...,...,...,...,...,...,...,...,...
2208737,2021-06-30,제주시,50110250,한림읍,W6XEB4,12,58250,1743
2208738,2021-06-30,제주시,50110250,한림읍,W6XEB7,15,111000,3326
2208739,2021-06-30,제주시,50110250,한림읍,W6XEB8,21,48300,1444
2208740,2021-06-30,제주시,50110250,한림읍,W6XEA2,5,22050,661


# 데이터 분할
train : -21.04, test : 21.05-06

In [ ]:
train = data2.copy()

In [ ]:
train['base_date'] = pd.to_datetime(train['base_date'],format="%Y-%m-%d")
train

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
0,2018-01-01,제주시,50110670,외도동,W6X062,2,15500,464
1,2018-01-01,제주시,50110630,아라동,W6XA97,25,59800,1787
2,2018-01-01,제주시,50110630,아라동,W6XA96,8,25350,758
3,2018-01-01,제주시,50110630,아라동,W6XA95,12,30000,898
4,2018-01-01,제주시,50110630,아라동,W6XA94,14,44050,1317
...,...,...,...,...,...,...,...,...
2208737,2021-06-30,제주시,50110250,한림읍,W6XEB4,12,58250,1743
2208738,2021-06-30,제주시,50110250,한림읍,W6XEB7,15,111000,3326
2208739,2021-06-30,제주시,50110250,한림읍,W6XEB8,21,48300,1444
2208740,2021-06-30,제주시,50110250,한림읍,W6XEA2,5,22050,661


### test(5월)

In [ ]:
test1 = train[(train['base_date'] >= '2021-05-01') & (train['base_date'] <= '2021-05-31')]
test1

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
2085014,2021-05-01,제주시,50110510,일도일동,W6X169,26,126200,3779
2085015,2021-05-01,제주시,50110250,한림읍,W6X16B,14,19400,578
2085016,2021-05-01,제주시,50110250,한림읍,W6X16C,4,8000,239
2085017,2021-05-01,제주시,50110250,한림읍,W6X16E,11,14050,420
2085018,2021-05-01,제주시,50110510,일도일동,W6X168,14,34250,1023
...,...,...,...,...,...,...,...,...
2147893,2021-05-31,제주시,50110670,외도동,W6X698,34,57750,1726
2147894,2021-05-31,제주시,50110670,외도동,W6X699,39,79350,2373
2147895,2021-05-31,제주시,50110670,외도동,W6X700,30,51800,1548
2147896,2021-05-31,제주시,50110670,외도동,W6X701,15,38150,1142


In [ ]:
test1 = month_resampling(test1)

100%|██████████| 41/41 [00:00<00:00, 154.71it/s]


In [ ]:
test1

### test(6월)

In [ ]:
test2 = train[(train['base_date'] >= '2021-06-01') & (train['base_date'] <= '2021-06-30')]
test2

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
2147898,2021-06-01,제주시,50110253,애월읍,W6XE71,10,12550,374
2147899,2021-06-01,제주시,50110253,애월읍,W6XE72,3,3700,110
2147900,2021-06-01,제주시,50110253,애월읍,W6XE73,12,26600,795
2147901,2021-06-01,제주시,50110253,애월읍,W6XE74,8,9650,288
2147902,2021-06-01,제주시,50110253,애월읍,W6XE75,26,72400,2168
...,...,...,...,...,...,...,...,...
2208737,2021-06-30,제주시,50110250,한림읍,W6XEB4,12,58250,1743
2208738,2021-06-30,제주시,50110250,한림읍,W6XEB7,15,111000,3326
2208739,2021-06-30,제주시,50110250,한림읍,W6XEB8,21,48300,1444
2208740,2021-06-30,제주시,50110250,한림읍,W6XEA2,5,22050,661


In [ ]:
test2 = month_resampling(test2)
test2

### train

In [ ]:
train = train[train['base_date'] < '2021-05-01']
train

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
0,2018-01-01,제주시,50110670,외도동,W6X062,2,15500,464
1,2018-01-01,제주시,50110630,아라동,W6XA97,25,59800,1787
2,2018-01-01,제주시,50110630,아라동,W6XA96,8,25350,758
3,2018-01-01,제주시,50110630,아라동,W6XA95,12,30000,898
4,2018-01-01,제주시,50110630,아라동,W6XA94,14,44050,1317
...,...,...,...,...,...,...,...,...
2085009,2021-04-30,제주시,50110670,외도동,W6X693,21,56050,1675
2085010,2021-04-30,제주시,50110670,외도동,W6X694,8,8550,254
2085011,2021-04-30,제주시,50110670,외도동,W6X695,25,37150,1107
2085012,2021-04-30,제주시,50110670,외도동,W6X696,13,29500,882


# 월 단위 다운 샘플링 함수를 정의

In [ ]:
def month_resampling(df):
    new_data = pd.DataFrame() 
    df['base_date'] = df['base_date'].astype(str)
    df['year_month'] = df['base_date'].str.slice(stop=7)
    year_month = df['year_month'].drop_duplicates()

    downsampling_data = df.groupby(['emd_nm', 'year_month']).em_g.sum()
    downsampling_data = pd.DataFrame(downsampling_data)
    downsampling_data = downsampling_data.reset_index(drop=False,inplace=False)
    
    for i in tqdm(df.emd_nm.unique()):
        dong = downsampling_data[downsampling_data['emd_nm']==i]

        dong = dong.merge(year_month,how='outer')

        dong = dong.sort_values(by=['year_month'], axis=0, ascending=True) 

        dong['emd_nm'] = dong['emd_nm'].fillna(i)
        
        new_data = pd.concat([new_data,dong],axis=0)
        
    return new_data

## 데이터를 월 단위로 다운 샘플링

In [ ]:
resampling_data = month_resampling(train)
resampling_data

100%|██████████| 41/41 [00:00<00:00, 183.19it/s]


,emd_nm,year_month,em_g
0,외도동,2018-01,95864700.0
1,외도동,2018-02,88582050.0
2,외도동,2018-03,97356250.0
3,외도동,2018-04,88844750.0
4,외도동,2018-05,94064300.0
...,...,...,...
13,한림읍,2020-12,65998800.0
14,한림읍,2021-01,60314250.0
15,한림읍,2021-02,60052534.0
16,한림읍,2021-03,69780550.0


# 평가척도
MAE = (1/n) * sum(abs(y - y_pred))

In [ ]:
def mae(df) :
  n = len(df['emd_nm'])
  df['dif'] = np.abs(df['em_g'] - df['em_g_pred'])
  result = sum(df['dif']) / n
  return result

# 1. 자기회귀누적이동평균 모델

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

pandas2ri.activate()

auto_arima = """
    function(ts){
        library(stats)
        library(forecast)
        d_params = ndiffs(ts)
        model = auto.arima(ts, max.p=2, d=d_params)
        forecasted_data = forecast(model, h=2)
        out_df = data.frame(forecasted_data$mean)
        colnames(out_df) = c('amount')
        out_df
    }
"""

auto_arima = robjects.r(auto_arima)
ts = robjects.r('ts')
c = robjects.r('c')

final_pred_1 = []
final_pred_2 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    print(i)

    train = ts(trash['em_g'], frequency=12) 

    forecast = auto_arima(train)
    final_pred_1.append(float(forecast.values[0]))
    final_pred_2.append(float(forecast.values[1]))
    dong.append(i)

  5%|▍         | 2/41 [00:00<00:03, 11.10it/s]

외도동
아라동
노형동


 10%|▉         | 4/41 [00:00<00:04,  8.83it/s]

도두동
오라동


 15%|█▍        | 6/41 [00:00<00:04,  7.54it/s]

...

용담일동
조천읍


 93%|█████████▎| 38/41 [00:04<00:00,  8.67it/s]

구좌읍
한경면


100%|██████████| 41/41 [00:04<00:00,  8.77it/s]

한림읍


### 5월 예측값 + 실제값

In [ ]:
dict_data5 = {'emd_nm':dong, 'em_g_pred':final_pred_1}
df5 = pd.DataFrame(dict_data5)

In [ ]:
df5 = df5.merge(test1, on='emd_nm')
df5

In [ ]:
mae(df5)

7514828.760884168

### 6월 예측값 + 실제값

In [ ]:
dict_data6 = {'emd_nm':dong, 'em_g_pred':final_pred_2}
df6 = pd.DataFrame(dict_data6)

In [ ]:
df6 = df6.merge(test2, on='emd_nm')
df6

In [ ]:
mae(df6)

9357841.673239227

# 2. 지수평활법

In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈

# pandas2ri를 활성화 
pandas2ri.activate()

ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""
# r() 함수로 r 자료형을 파이썬에서 사용 가능
ets = robjects.r(ets)# str 형식으로 정의된 ets
ts = robjects.r('ts')# r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수

final_pred_1 = []
final_pred_2 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]

    # R의 ts 함수로 time series 데이터로 변환
    train = ts(trash['em_g'], frequency=12) 
    # 지수평활법l
    forecast = ets(train)

    final_pred_1.append(float(forecast.values[0]))
    final_pred_2.append(float(forecast.values[1]))
    dong.append(i)

100%|██████████| 41/41 [00:00<00:00, 61.74it/s]


## 5월 예측값 + 실제값

In [ ]:
dict_data5_2 = {'emd_nm':dong, 'em_g_pred':final_pred_1}
df5_2 = pd.DataFrame(dict_data5_2)

In [ ]:
df5_2 = df5_2.merge(test1, on='emd_nm')

In [ ]:
df5_2

,emd_nm,em_g_pred,year_month,em_g,dif
0,외도동,9.457219e+07,2021-05,97421950,2.849760e+06
1,아라동,9.712932e+07,2021-05,107045700,9.916381e+06
2,노형동,2.185667e+08,2021-05,227061100,8.494359e+06
3,도두동,1.733808e+07,2021-05,19663550,2.325467e+06
4,오라동,5.149242e+07,2021-05,57321800,5.829384e+06
5,삼양동,8.508381e+07,2021-05,93122870,8.039055e+06
6,이도이동,1.929509e+08,2021-05,215882550,2.293161e+07
7,일도이동,1.317045e+08,2021-05,147183350,1.547881e+07
8,건입동,3.506746e+07,2021-05,38047500,2.980043e+06
9,화북동,9.843370e+07,2021-05,109316800,1.088310e+07


In [ ]:
mae(df5_2)

5152619.267989604

## 6월 예측값 + 실제값

In [ ]:
dict_data6_2 = {'emd_nm':dong, 'em_g_pred':final_pred_2}
df6_2 = pd.DataFrame(dict_data6_2)

In [ ]:
df6_2 = df6_2.merge(test2, on='emd_nm')

In [ ]:
df6_2

In [ ]:
mae(df6_2)

6955079.73237115

# 3. STL 분해를 적용한 지수평활법

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈

# pandas2ri를 활성화 
pandas2ri.activate()
stlm = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = stlm(ts, s.window="periodic", method='ets') # STL 분해 후 지수평활법을 통한 예측 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""
ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""
# r() 함수로 r을 파이썬에서 사용 가능
stlm = robjects.r(stlm)# str 형식으로 정의된 stlm
ets = robjects.r(ets)# str 형식으로 정의된 ets
ts = robjects.r('ts')# r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수

final_pred_1 = []
final_pred_2 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    data_len = len(trash)
    
    # R의 ts 함수로 time series 데이터로 변환
    train = ts(trash['em_g'], frequency=12) 
    # STL 분해를 적용한 지수평활 model
    if data_len > 24:
        forecast = stlm(train)
    # 지수평활 model
    else:
        forecast = ets(train)
    final_pred_1.append(float(forecast.values[0]))
    final_pred_2.append(float(forecast.values[1]))
    dong.append(i)

# 4. 로그정규화(자기회귀누적이동평균 모델)

In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
import numpy as np

# pandas2ri를 활성화 
pandas2ri.activate()

auto_arima = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        d_params = ndiffs(ts) # 시계열 자료의 차분 횟수 계산
        model = auto.arima(ts, max.p=2, d=d_params) # auto.arima 모델 생성
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""

# r() 함수로 r 자료형을 파이썬에서 사용 가능
auto_arima = robjects.r(auto_arima)
ts = robjects.r('ts')# r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log')# 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수


final_pred_1 = []
final_pred_2 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    print(i)

    train = ts(log(trash['em_g']), frequency=12) 

    forecast = auto_arima(train)
    final_pred_1.append(float(exp(forecast.values[0])))
    final_pred_2.append(float(exp(forecast.values[1])))
    dong.append(i)

  5%|▍         | 2/41 [00:00<00:03, 10.55it/s]

외도동
아라동
노형동


 10%|▉         | 4/41 [00:00<00:03, 10.14it/s]

도두동
오라동
삼양동


 15%|█▍        | 6/41 [00:00<00:03,  9.75it/s]

...

삼도이동
일도일동


 93%|█████████▎| 38/41 [00:04<00:00,  7.57it/s]

용담일동
조천읍
구좌읍


100%|██████████| 41/41 [00:04<00:00,  8.43it/s]

한경면
한림읍


## 5월 예측값 + 실제값

In [ ]:
dict_data5_4 = {'emd_nm':dong, 'em_g_pred':final_pred_1}
df5_4 = pd.DataFrame(dict_data5_4)

In [ ]:
df5_4 = df5_4.merge(test1, on='emd_nm')

In [ ]:
df5_4

,emd_nm,em_g_pred,year_month,em_g
0,외도동,9.301077e+07,2021-05,97421950
1,아라동,9.468290e+07,2021-05,107045700
2,노형동,2.128510e+08,2021-05,227061100
3,도두동,1.717795e+07,2021-05,19663550
4,오라동,5.523606e+07,2021-05,57321800
5,삼양동,8.482984e+07,2021-05,93122870
6,이도이동,1.925272e+08,2021-05,215882550
7,일도이동,1.314622e+08,2021-05,147183350
8,건입동,3.538685e+07,2021-05,38047500
9,화북동,1.056905e+08,2021-05,109316800


In [ ]:
mae(df5_4)

5032194.224844982

## 6월 예측값 + 실제값

In [ ]:
dict_data6_4 = {'emd_nm':dong, 'em_g_pred':final_pred_2}
df6_4 = pd.DataFrame(dict_data6_4)

In [ ]:
df6_4 = df6_4.merge(test2, on='emd_nm')

In [ ]:
df6_4

,emd_nm,em_g_pred,year_month,em_g
0,외도동,9.563415e+07,2021-06,100096900
1,아라동,9.468290e+07,2021-06,112190800
2,노형동,2.128510e+08,2021-06,239069250
3,도두동,1.706341e+07,2021-06,19733350
4,오라동,5.899631e+07,2021-06,58187300
5,삼양동,8.482984e+07,2021-06,96172500
6,이도이동,1.925272e+08,2021-06,225423200
7,일도이동,1.314622e+08,2021-06,147438200
8,건입동,3.538685e+07,2021-06,36493300
9,화북동,1.085403e+08,2021-06,110750050


In [ ]:
mae(df6_4)

7189293.242274136

# 5. 로그정규화(지수평활법)

In [ ]:
# 매출 변동 계수를 구하는 함수
def coefficient_variation(df, i):
    cv_data = df.groupby(['emd_nm']).em_g.std()/df.groupby(['emd_nm']).em_g.mean()
    cv = cv_data[i]
    return cv

In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
import numpy as np

# pandas2ri를 활성화 
pandas2ri.activate()

ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""

# r() 함수로 r 자료형을 파이썬에서 사용 가능
ets = robjects.r(ets)
ts = robjects.r('ts') # r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log') # 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수

final_pred_1 = []
final_pred_2 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    
    cv = coefficient_variation(resampling_data, i)
    # 매출액 변동 계수가 0.3 미만인 경우만 log를 씌움
    if cv < 0.3:
        train_log = ts(log(trash['em_g']), frequency=12) 
        # ets model
        forecast_log = ets(train_log)
        final_pred_1.append(float(exp(forecast_log.values[0])))
        final_pred_2.append(float(exp(forecast_log.values[1])))
        dong.append(i)
    # 매출액 변동 계수가 0.3 이상인 경우
    else:
        train = ts(trash['em_g'], frequency=12)
        # 지수평활법
        forecast = ets(train)
        final_pred_1.append(float(forecast.values[0]))
        final_pred_2.append(float(forecast.values[1]))
        dong.append(i)

100%|██████████| 41/41 [00:01<00:00, 34.64it/s]


## 5월 예측값 + 실제값

In [ ]:
dict_data5_5 = {'emd_nm':dong, 'em_g_pred':final_pred_1}
df5_5 = pd.DataFrame(dict_data5_5)

In [ ]:
df5_5 = df5_5.merge(test1, on='emd_nm')

In [ ]:
df5_5

,emd_nm,em_g_pred,year_month,em_g
0,외도동,9.566962e+07,2021-05,97421950
1,아라동,9.868538e+07,2021-05,107045700
2,노형동,2.362257e+08,2021-05,227061100
3,도두동,1.731328e+07,2021-05,19663550
4,오라동,5.158196e+07,2021-05,57321800
5,삼양동,8.762481e+07,2021-05,93122870
6,이도이동,2.048939e+08,2021-05,215882550
7,일도이동,1.399111e+08,2021-05,147183350
8,건입동,3.491084e+07,2021-05,38047500
9,화북동,9.955116e+07,2021-05,109316800


In [ ]:
mae(df5_5)

4353197.60184052

## 6월 예측값 + 실제값

In [ ]:
dict_data6_5 = {'emd_nm':dong, 'em_g_pred':final_pred_2}
df6_5 = pd.DataFrame(dict_data6_5)

In [ ]:
df6_5 = df6_5.merge(test2, on='emd_nm')

In [ ]:
df6_5

,emd_nm,em_g_pred,year_month,em_g
0,외도동,9.566962e+07,2021-06,100096900
1,아라동,9.868538e+07,2021-06,112190800
2,노형동,2.354890e+08,2021-06,239069250
3,도두동,1.731328e+07,2021-06,19733350
4,오라동,5.158196e+07,2021-06,58187300
5,삼양동,8.762481e+07,2021-06,96172500
6,이도이동,2.037468e+08,2021-06,225423200
7,일도이동,1.390659e+08,2021-06,147438200
8,건입동,3.491084e+07,2021-06,36493300
9,화북동,9.955116e+07,2021-06,110750050


In [ ]:
mae(df6_5)

5768645.296802302

# 6. forecastHybrid를 통한 앙상블

In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
import numpy as np

# pandas2ri를 활성화 
pandas2ri.activate()

hybridModel = """
    function(ts){
        library(forecast)
        library(forecastHybrid)
        d_params=ndiffs(ts)
        hb_mdl<-hybridModel(ts, models="aes", # auto_arima, ets, stlm
                        a.arg=list(max.p=2, d=d_params), # auto_arima parameter
                        weight="equal") # 가중치를 동일하게 줌(평균)
        forecasted_data<-forecast(hb_mdl, h=2) # 이후 2개월(h=2)을 예측
        outdf<-data.frame(forecasted_data$mean)
        colnames(outdf)<-c('amount')
        outdf
    }
""" 

# r() 함수로 r 자료형을 파이썬에서 사용 가능
hybridModel = robjects.r(hybridModel)
ts = robjects.r('ts') # r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log') # 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수

final_pred_1 = []
final_pred_2 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    
    cv = coefficient_variation(resampling_data, i)
    # 매출액 변동 계수가 0.3 미만인 경우만 log를 씌움
    if cv < 0.3:
        train_log = ts(log(trash['em_g']), frequency=12) 
        # 앙상블 예측
        forecast_log = hybridModel(train_log)
        final_pred_1.append(float(exp(forecast_log.values[0])))
        final_pred_2.append(float(exp(forecast_log.values[1])))
        dong.append(i)
    # 매출액 변동 계수가 0.3 이상인 경우
    else:
        train = ts(trash['em_g'], frequency=12)
        # 앙상블 예측
        forecast = hybridModel(train)
        final_pred_1.append(float(forecast.values[0]))
        final_pred_2.append(float(forecast_log.values[1]))
        dong.append(i)

## 5월 예측값 + 실제값

In [ ]:
dict_data5_6 = {'emd_nm':dong, 'em_g_pred':final_pred_1}
df5_6 = pd.DataFrame(dict_data5_6)

In [ ]:
df5_6 = df5_6.merge(test1, on='emd_nm')

In [ ]:
df5_6

,emd_nm,em_g_pred,year_month,em_g
0,외도동,9.387543e+07,2021-05,97421950
1,아라동,9.590611e+07,2021-05,107045700
2,노형동,2.157089e+08,2021-05,227061100
3,도두동,1.727726e+07,2021-05,19663550
4,오라동,5.586915e+07,2021-05,57321800
5,삼양동,8.495683e+07,2021-05,93122870
6,이도이동,1.927391e+08,2021-05,215882550
7,일도이동,1.315834e+08,2021-05,147183350
8,건입동,3.522715e+07,2021-05,38047500
9,화북동,1.081963e+08,2021-05,109316800


In [ ]:
mae(df5_6)

5240255.55834417

## 6월 예측값 + 실제값

In [ ]:
dict_data6_6 = {'emd_nm':dong, 'em_g_pred':final_pred_2}
df6_6 = pd.DataFrame(dict_data6_6)

In [ ]:
df6_6 = df6_6.merge(test2, on='emd_nm')

In [ ]:
df6_6

,emd_nm,em_g_pred,year_month,em_g
0,외도동,9.528610e+07,2021-06,100096900
1,아라동,9.590611e+07,2021-06,112190800
2,노형동,2.157089e+08,2021-06,239069250
3,도두동,1.723158e+07,2021-06,19733350
4,오라동,5.481251e+07,2021-06,58187300
5,삼양동,8.495683e+07,2021-06,96172500
6,이도이동,1.927391e+08,2021-06,225423200
7,일도이동,1.315834e+08,2021-06,147438200
8,건입동,3.522715e+07,2021-06,36493300
9,화북동,1.047347e+08,2021-06,110750050


In [ ]:
mae(df6_6)

7467003.029259603

In [ ]:
len(final_pred_1)

37

# 0. 로그정규화(지수평활법) > 7,8월 예측

## 데이터를 월 단위로 다운 샘플링

In [ ]:
resampling_data = month_resampling(data2)
resampling_data

100%|██████████| 41/41 [00:00<00:00, 142.49it/s]


,emd_nm,year_month,em_g
0,외도동,2018-01,95864700.0
1,외도동,2018-02,88582050.0
2,외도동,2018-03,97356250.0
3,외도동,2018-04,88844750.0
4,외도동,2018-05,94064300.0
...,...,...,...
15,한림읍,2021-02,60052534.0
16,한림읍,2021-03,69780550.0
17,한림읍,2021-04,73755600.0
18,한림읍,2021-05,79295550.0


In [ ]:
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
import numpy as np

# pandas2ri를 활성화 
pandas2ri.activate()

ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""

# r() 함수로 r 자료형을 파이썬에서 사용 가능
ets = robjects.r(ets)
ts = robjects.r('ts') # r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log') # 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수

final_pred_1 = []
final_pred_2 = []
dong = []

for i in tqdm(resampling_data.emd_nm.unique()):
    trash = resampling_data[resampling_data['emd_nm']==i]
    
    cv = coefficient_variation(resampling_data, i)
    # 매출액 변동 계수가 0.3 미만인 경우만 log를 씌움
    if cv < 0.3:
        train_log = ts(log(trash['em_g']), frequency=12) 
        # ets model
        forecast_log = ets(train_log)
        final_pred_1.append(float(exp(forecast_log.values[0])))
        final_pred_2.append(float(exp(forecast_log.values[1])))
        dong.append(i)
    # 매출액 변동 계수가 0.3 이상인 경우
    else:
        train = ts(trash['em_g'], frequency=12)
        # 지수평활법
        forecast = ets(train)
        final_pred_1.append(float(forecast.values[0]))
        final_pred_2.append(float(forecast.values[1]))
        dong.append(i)

100%|██████████| 41/41 [00:00<00:00, 44.19it/s]


## 7월 예측값

In [ ]:
dict_data7 = {'emd_nm':dong, 'em_g_pred':final_pred_1}
df7 = pd.DataFrame(dict_data7)
df7

,emd_nm,em_g_pred
0,외도동,9.567022e+07
1,아라동,1.077801e+08
2,노형동,2.318757e+08
3,도두동,1.960046e+07
4,오라동,5.807474e+07
5,삼양동,9.129482e+07
6,이도이동,2.056412e+08
7,일도이동,1.397760e+08
8,건입동,3.667725e+07
9,화북동,1.054530e+08


## 8월 예측값

In [ ]:
dict_data8 = {'emd_nm':dong, 'em_g_pred':final_pred_2}
df8 = pd.DataFrame(dict_data8)
df8

,emd_nm,em_g_pred
0,외도동,9.567022e+07
1,아라동,1.077801e+08
2,노형동,2.310044e+08
3,도두동,1.960046e+07
4,오라동,5.807474e+07
5,삼양동,9.129482e+07
6,이도이동,2.045675e+08
7,일도이동,1.389756e+08
8,건입동,3.667725e+07
9,화북동,1.054530e+08


In [ ]:
dict_data = {'emd_nm':dong, 'em_g_pred_7':final_pred_1, 'em_g_pred_8':final_pred_2}
df_final = pd.DataFrame(dict_data)
df_final

,emd_nm,em_g_pred_7,em_g_pred_8
0,외도동,9.567022e+07,9.567022e+07
1,아라동,1.077801e+08,1.077801e+08
2,노형동,2.318757e+08,2.310044e+08
3,도두동,1.960046e+07,1.960046e+07
4,오라동,5.807474e+07,5.807474e+07
5,삼양동,9.129482e+07,9.129482e+07
6,이도이동,2.056412e+08,2.045675e+08
7,일도이동,1.397760e+08,1.389756e+08
8,건입동,3.667725e+07,3.667725e+07
9,화북동,1.054530e+08,1.054530e+08


In [ ]:
df_final.to_csv("/content/drive/MyDrive/세미2차/78월시계열예측.csv", index=False, encoding='CP949')